# Making a Face Detector using VGG16

### Loading the model

In [1]:
from keras.applications import VGG16

img_rows = 224
img_cols = 224

model = VGG16(weights = 'imagenet', include_top = False, input_shape = (img_rows,img_cols,3))

Using TensorFlow backend.


## Freezing the Layers

In [2]:
for i in model.layers:
    i.trainable=False
    print(i.__class__.__name__,'=',i.trainable)

InputLayer = False
Conv2D = False
Conv2D = False
MaxPooling2D = False
Conv2D = False
Conv2D = False
MaxPooling2D = False
Conv2D = False
Conv2D = False
Conv2D = False
MaxPooling2D = False
Conv2D = False
Conv2D = False
Conv2D = False
MaxPooling2D = False
Conv2D = False
Conv2D = False
Conv2D = False
MaxPooling2D = False


## Creating the top Model

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


In [4]:
top = model.output
top = Flatten(name = "flatten")(top)
top = Dense(512,activation='relu')(top)
top = Dense(256,activation='relu')(top)
top = Dropout(0.3)(top)
top = Dense(14,activation='softmax')(top)

## Checking the Top and Bottom Models layers
### Joing them to one single model

In [5]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [6]:
model.layers[:10]

In [7]:
model = Model(inputs = model.input, outputs = top)

In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Loading dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'D:/books and other college files/Vimal sir trainings/MLOps/Tasks/Face Detection using VGG16/14-celebrity-faces-dataset/data/train/'
validation_data_dir = 'D:/books and other college files/Vimal sir trainings/MLOps/Tasks/Face Detection using VGG16/14-celebrity-faces-dataset/data/val/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

## Training to Layers

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("FaceDetect.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]


model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 20,
    callbacks=callbacks,
    validation_data = validation_generator,
    validation_steps =70//7)

# Saving Model

In [ ]:
model.save("FaceDetect.h5")

## Using the model for Prediction

In [9]:
from keras.models import load_model

classifier = load_model('FaceDetect.h5')

In [10]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face_detect_dict = {"[0]":"anne_hathaway", 
                      "[1]":"arnold_schwarzenegger",
                      "[2]":"ben_afflek",
                      "[3]":"dwayne_johnson",
                      "[4]":"elton_john",
                      "[5]":"jerry_seinfeld",
                      "[6]":"kate_beckinsale",
                      "[7]":"keanu_reeves",
                      "[8]":"lauren_cohan",
                      "[9]":"madonna",
                      "[10]":"mindy_kaling",
                      "[11]":"simon_pegg",
                      "[12]":"sofia_vergara",
                      "[13]":"will_smith"}

face_detect_dict_n = {"anne_hathaway":"anne_hathaway", 
                      "arnold_schwarzenegger":"arnold_schwarzenegger",
                      "ben_afflek":"ben_afflek",
                      "dwayne_johnson":"dwayne_johnson",
                      "elton_john":"elton_john",
                      "jerry_seinfeld":"jerry_seinfeld",
                      "kate_beckinsale":"kate_beckinsale",
                      "keanu_reeves":"keanu_reeves",
                      "lauren_cohan":"lauren_cohan",
                      "madonna":"madonna",
                      "mindy_kaling":"mindy_kaling",
                      "simon_pegg":"simon_pegg",
                      "sofia_vergara":"sofia_vergara",
                      "will_smith":"will_smith"}

def draw_test(name, pred, im):
    face = face_detect_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_detect_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,20):
    input_im = getRandomImage("14-celebrity-faces-dataset/data/val/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    print(classifier.predict(input_im, 1, verbose = 0))
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)
cv2.destroyAllWindows()

Class - anne_hathaway
[[0.13312018 0.026843   0.02829136 0.12008972 0.00529452 0.0081012
  0.0169046  0.00894157 0.05892804 0.00386109 0.36613595 0.02931349
  0.0297173  0.16445802]]
Class - kate_beckinsale
[[0.05740617 0.15776706 0.02047782 0.033091   0.04691602 0.04561999
  0.01663455 0.00991363 0.17593582 0.00976022 0.24368869 0.04028679
  0.04705974 0.09544252]]
Class - will_smith
[[0.0057909  0.45707163 0.03475057 0.01920632 0.0269912  0.02423453
  0.02129125 0.00433596 0.00412695 0.00596067 0.00952011 0.08466593
  0.00166286 0.3003911 ]]
Class - simon_pegg
[[0.01673489 0.37116548 0.0733412  0.03797368 0.01283374 0.011239
  0.07333936 0.01799204 0.01785216 0.00755152 0.0197725  0.07118068
  0.00356826 0.26545542]]
Class - madonna
[[8.2089187e-04 3.6260295e-07 8.6516293e-06 4.2283709e-06 4.3504599e-05
  8.1581238e-05 8.4569072e-04 3.1470041e-05 5.9768389e-04 9.9416590e-01
  7.3220575e-04 1.8602437e-05 2.6240498e-03 2.5265554e-05]]
Class - keanu_reeves
[[0.0486947  0.18139376 0.0251